# penalty form repair without mail

In [2]:
# importing libraries

import pandas as pd
from pandas import Series,DataFrame
import datetime
from datetime import date, timedelta
import pygsheets
import warnings
import numpy as np
warnings.filterwarnings("ignore")

clients = pygsheets.authorize(service_file='repair.json')

allotment_status_report='1cpR6AVVpk9TF4_I38IFYPPOqk-_bSROHgVVYdaXLXOI'
car_no_master_list='11D8_6u4ywy3yNYyrnMonyti6eflqUtpMfsxgLMBXEzk'
penalty_form_repair = '1Txa0MVP7Kxxjhwjd5EwME3JfEPSi5q2klC7GuZw2hRc'
testing='1ELkL-R5Hs2FhOA7huIirzfebLxX57cN2v3wFQnKrupY'
#car no master list

car_no_master_list_sheet= clients.open_by_key(car_no_master_list)
ws= car_no_master_list_sheet.worksheet_by_title('History')
data = ws.get_all_values()
headers = data.pop(0)
car_no_master_list = pd.DataFrame(data,columns=headers)
car_no_master_list=car_no_master_list[['Date', 'Car No','DM','End']]
car_no_master_list['Car No']=car_no_master_list['Car No'].str.replace(' ','')
car_no_master_list['Date']=pd.to_datetime(car_no_master_list['Date'], errors='coerce',format='%d-%b-%Y')
car_no_master_list['End']=pd.to_datetime(car_no_master_list['End'], errors='coerce',format='%d/%m/%Y')
car_no_master_list.rename({'Car No':'Car Number'},axis=1,inplace=True)


#allotment status report sheet

sheet = clients.open_by_key(allotment_status_report)
ws = sheet.worksheet_by_title('New Allotment History')
data = ws.get_all_values()
headers = data.pop(0)
allotment_s_r = pd.DataFrame(data,columns=headers)
allotment_s_r = allotment_s_r[['Allotment Date','Return Date','Car Number','ETM','DM Name']]
allotment_s_r['Car Number']=allotment_s_r['Car Number'].str.replace(' ','')
today = datetime.datetime.now()
today_timestmp_column = today.strftime("%d/%m/%Y %H:%M:%S")
allotment_s_r['Allotment Date']=allotment_s_r['Allotment Date'].replace('',(today_timestmp_column))
allotment_s_r['Return Date']=allotment_s_r['Return Date'].replace('',(today_timestmp_column))
allotment_s_r[['Allotment Date','Return Date']]=allotment_s_r[['Allotment Date','Return Date']].apply(pd.to_datetime, errors='coerce',format='%d/%m/%Y %H:%M:%S')


#penalty form sheet

sheet= clients.open_by_key(penalty_form_repair)
ws= sheet.worksheet_by_title('Form responses')
data = ws.get_all_values()
headers = data.pop(0)
responses_df = pd.DataFrame(data,columns=headers)
responses_df=responses_df[['Timestamp', 'Car Number', 'Panel Types']]
responses_df['Car Number']=responses_df['Car Number'].str.replace(' ','')
responses_df['Timestamp']=pd.to_datetime(responses_df['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')

#merging allotment 

allotment_car_no=responses_df.merge(allotment_s_r, on='Car Number',how='left')
allotment_car_no['Timestamp']=pd.to_datetime(allotment_car_no['Timestamp']).dt.strftime('%Y-%m-%d')
allotment_car_no_df = allotment_car_no[(allotment_car_no['Allotment Date'] <= allotment_car_no['Timestamp']) & (allotment_car_no['Timestamp'] <= allotment_car_no['Return Date'])]
allotment_car_no_df=allotment_car_no_df[['Car Number','ETM']]
allotment_car_no_df
    
#merging car_no_master_list

car_no_master_dm_name=responses_df.merge(car_no_master_list, on='Car Number',how='left')
car_no_master_dm_name_df = car_no_master_dm_name[(car_no_master_dm_name['Date'] <= car_no_master_dm_name['Timestamp']) & (car_no_master_dm_name['Timestamp'] <= car_no_master_dm_name['End'])]
car_no_master_dm_name_df=car_no_master_dm_name_df[['Car Number','DM']]

#merging allotment_and_car_no

allotment_etm_and_car_no_master=allotment_car_no_df.merge(car_no_master_dm_name_df, on='Car Number', how='left')

#penalty form sheet


responses_df_new = responses_df['Panel Types'].str.split(',').apply(Series,1).stack()

print(responses_df_new.head(10))

responses_df_new.index = responses_df_new.index.droplevel(-1)

responses_df_new.name = 'Panel Types'

print(responses_df_new.head())

del responses_df['Panel Types']

amount_final = responses_df.join(responses_df_new)

print(amount_final)

amount_final_df=amount_final.merge(allotment_etm_and_car_no_master, on='Car Number',how='left')

amount_final_df[['Panel name','Amount']]= amount_final_df['Panel Types'].str.split('-',expand=True)



print(amount_final_df)

del amount_final_df['Panel Types']

amount_final_df['Timestamp']=amount_final_df['Timestamp'].astype(str)
amount_final_df['Timestamp'].replace('NaT','', inplace=True,regex=True)

amount_final_df.drop_duplicates(inplace=True)

# amount_final_df['ETM'].fillna('',inplace=True)
# amount_final_df['DM'].fillna('',inplace=True)
# amount_final_df['Amount'].fillna('',inplace=True)

# amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Leasing','')
# amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Insurance','')
# amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Repairing','')
# amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Vinay Bhai','')

# amount_final_df['DM']=amount_final_df['DM'].str.replace('Deadly Shark','')
# amount_final_df['DM']=amount_final_df['DM'].str.replace('Red Eagle','')
# amount_final_df['DM']=amount_final_df['DM'].str.replace('Vinay Bhai Leasing','')
# amount_final_df['DM']=amount_final_df['DM'].str.replace('Vinay Bhai','')


car_no_master_list_dm_tab= car_no_master_list_sheet.worksheet_by_title('DM')
data = car_no_master_list_dm_tab.get_all_values()
headers = data.pop(0)
car_no_master_list_dm_data = pd.DataFrame(data,columns=headers)
mumbai_60_40=car_no_master_list_dm_data[car_no_master_list_dm_data['Cities']=='Mumbai 60:40']

amount_final_df=amount_final_df[amount_final_df['DM'].isin(mumbai_60_40['DM NAME'])]
amount_final_df


print(amount_final_df.head())
print(amount_final_df.shape)

# amount_final_df['DM'] = np.where(
#     amount_final_df['ETM'] == '',
#     '',
#     amount_final_df['DM']
# )        

sheet= clients.open_by_key(penalty_form_repair)
ws= sheet.worksheet_by_title('Penalty_amount')
ws.clear()
ws.set_dataframe(amount_final_df,(1,1))

0  0           Bonnet-300
1  0     Front Bumper-300
2  0     Front Bumper-300
3  0    Front Bumper-300 
   1      LHS fender-300 
4  0            Dicky-300
5  0     Front Bumper-300
6  0           Bonnet-300
   1     Front Bumper-300
7  0    LHS Rear door-300
dtype: object
0           Bonnet-300
1     Front Bumper-300
2     Front Bumper-300
3    Front Bumper-300 
3      LHS fender-300 
Name: Panel Types, dtype: object
              Timestamp  Car Number        Panel Types
0   2022-07-06 18:45:57  MH01DR2761         Bonnet-300
1   2022-07-07 16:34:59  MH01DR5671   Front Bumper-300
2   2022-07-07 19:21:06  MH01CV7902   Front Bumper-300
3   2022-07-08 12:10:36  MH01DR7075  Front Bumper-300 
3   2022-07-08 12:10:36  MH01DR7075    LHS fender-300 
..                  ...         ...                ...
157                 NaT                               
158                 NaT                               
159                 NaT                               
160                 NaT     

# with mail

In [ ]:
import os, sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
from mail.send_mail import sendMail
import pandas as pd
from pandas import Series,DataFrame
import datetime
from datetime import date, timedelta
import pygsheets
import warnings
import numpy as np
warnings.filterwarnings("ignore")

clients = pygsheets.authorize(service_file='/home/karan/config/repair.json')

try:
    #sheet keys
    
    allotment_status_report='1cpR6AVVpk9TF4_I38IFYPPOqk-_bSROHgVVYdaXLXOI'
    car_no_master_list='11D8_6u4ywy3yNYyrnMonyti6eflqUtpMfsxgLMBXEzk'
    penalty_form_repair = '1Txa0MVP7Kxxjhwjd5EwME3JfEPSi5q2klC7GuZw2hRc'
    
    #car no master list

    car_no_master_list_sheet= clients.open_by_key(car_no_master_list)
    ws= car_no_master_list_sheet.worksheet_by_title('History')
    data = ws.get_all_values()
    headers = data.pop(0)
    car_no_master_list = pd.DataFrame(data,columns=headers)
    car_no_master_list=car_no_master_list[['Date', 'Car No','DM','End']]
    car_no_master_list['Car No']=car_no_master_list['Car No'].str.replace(' ','')
    car_no_master_list['Date']=pd.to_datetime(car_no_master_list['Date'], errors='coerce',format='%d-%b-%Y')
    car_no_master_list['End']=pd.to_datetime(car_no_master_list['End'], errors='coerce',format='%d/%m/%Y')
    car_no_master_list.rename({'Car No':'Car Number'},axis=1,inplace=True)


    #allotment status report sheet

    sheet = clients.open_by_key(allotment_status_report)
    ws = sheet.worksheet_by_title('New Allotment History')
    data = ws.get_all_values()
    headers = data.pop(0)
    allotment_s_r = pd.DataFrame(data,columns=headers)
    allotment_s_r = allotment_s_r[['Allotment Date','Return Date','Car Number','ETM','DM Name']]
    allotment_s_r['Car Number']=allotment_s_r['Car Number'].str.replace(' ','')
    today = datetime.datetime.now()
    today_timestmp_column = today.strftime("%d/%m/%Y %H:%M:%S")
    allotment_s_r['Allotment Date']=allotment_s_r['Allotment Date'].replace('',(today_timestmp_column))
    allotment_s_r['Return Date']=allotment_s_r['Return Date'].replace('',(today_timestmp_column))
    allotment_s_r[['Allotment Date','Return Date']]=allotment_s_r[['Allotment Date','Return Date']].apply(pd.to_datetime, errors='coerce',format='%d/%m/%Y %H:%M:%S')


    #penalty form sheet

    sheet= clients.open_by_key(penalty_form_repair)
    ws= sheet.worksheet_by_title('Form responses')
    data = ws.get_all_values()
    headers = data.pop(0)
    responses_df = pd.DataFrame(data,columns=headers)
    responses_df=responses_df[['Timestamp', 'Car Number', 'Panel Types']]
    responses_df['Car Number']=responses_df['Car Number'].str.replace(' ','')
    responses_df['Timestamp']=pd.to_datetime(responses_df['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')

    #merging allotment 

    allotment_car_no=responses_df.merge(allotment_s_r, on='Car Number',how='left')
    allotment_car_no['Timestamp']=pd.to_datetime(allotment_car_no['Timestamp']).dt.strftime('%Y-%m-%d')
    allotment_car_no_df = allotment_car_no[(allotment_car_no['Allotment Date'] <= allotment_car_no['Timestamp']) & (allotment_car_no['Timestamp'] <= allotment_car_no['Return Date'])]
    allotment_car_no_df=allotment_car_no_df[['Car Number','ETM']]
    allotment_car_no_df

    #merging car_no_master_list

    car_no_master_dm_name=responses_df.merge(car_no_master_list, on='Car Number',how='left')
    car_no_master_dm_name_df = car_no_master_dm_name[(car_no_master_dm_name['Date'] <= car_no_master_dm_name['Timestamp']) & (car_no_master_dm_name['Timestamp'] <= car_no_master_dm_name['End'])]
    car_no_master_dm_name_df=car_no_master_dm_name_df[['Car Number','DM']]

    #merging allotment_and_car_no

    allotment_etm_and_car_no_master=allotment_car_no_df.merge(car_no_master_dm_name_df, on='Car Number', how='left')

    #penalty form sheet


    responses_df_new = responses_df['Panel Types'].str.split(',').apply(Series,1).stack()

    print(responses_df_new.head(10))

    responses_df_new.index = responses_df_new.index.droplevel(-1)

    responses_df_new.name = 'Panel Types'

    print(responses_df_new.head())

    del responses_df['Panel Types']

    amount_final = responses_df.join(responses_df_new)

    print(amount_final)

    amount_final_df=amount_final.merge(allotment_etm_and_car_no_master, on='Car Number',how='left')

    amount_final_df[['Panel name','Amount']]= amount_final_df['Panel Types'].str.split('-',expand=True)


    print(amount_final_df)

    del amount_final_df['Panel Types']

    amount_final_df['Timestamp']=amount_final_df['Timestamp'].astype(str)
    amount_final_df['Timestamp'].replace('NaT','', inplace=True,regex=True)

    amount_final_df.drop_duplicates(inplace=True)

    # amount_final_df['ETM'].fillna('',inplace=True)
    # amount_final_df['DM'].fillna('',inplace=True)
    # amount_final_df['Amount'].fillna('',inplace=True)

    # amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Leasing','')
    # amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Insurance','')
    # amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Repairing','')
    # amount_final_df['ETM']=amount_final_df['ETM'].str.replace('Vinay Bhai','')

    # amount_final_df['DM']=amount_final_df['DM'].str.replace('Deadly Shark','')
    # amount_final_df['DM']=amount_final_df['DM'].str.replace('Red Eagle','')
    # amount_final_df['DM']=amount_final_df['DM'].str.replace('Vinay Bhai Leasing','')
    # amount_final_df['DM']=amount_final_df['DM'].str.replace('Vinay Bhai','')


    car_no_master_list_dm_tab= car_no_master_list_sheet.worksheet_by_title('DM')
    data = car_no_master_list_dm_tab.get_all_values()
    headers = data.pop(0)
    car_no_master_list_dm_data = pd.DataFrame(data,columns=headers)
    mumbai_60_40=car_no_master_list_dm_data[car_no_master_list_dm_data['Cities']=='Mumbai 60:40']

    amount_final_df=amount_final_df[amount_final_df['DM'].isin(mumbai_60_40['DM NAME'])]
    amount_final_df


    print(amount_final_df.head())
    print(amount_final_df.shape)

    # amount_final_df['DM'] = np.where(
    #     amount_final_df['ETM'] == '',
    #     '',
    #     amount_final_df['DM']
    # )        

    sheet= clients.open_by_key(penalty_form_repair)
    ws= sheet.worksheet_by_title('Penalty_amount')
    ws.clear()
    ws.set_dataframe(amount_final_df,(1,1))
except Exception: 
    sendMail(["abusalameverestfleet@gmail.com",
              "avinash.everestfleet@gmail.com",
              "rutvijaeverestfleet@gmail.com",
              "karan.saraogi@everestfleet.com"
              "sameer.everestfleet@gmail.com"])
print("Penalty_amount updated succesfully")

In [48]:
responses_df[32:40]

,Timestamp,Car Number,Panel Types
32,2022-07-22 13:23:41,MH01DR2870,"Front Bumper-300, RHS fender-300, RHS Front do..."
33,2022-07-22 16:57:59,MH01DR2547,"Front Bumper-300, RHS fender-300, Rear Bumper-300"
34,2022-07-23 11:19:29,MH01CV8116,"Front Bumper-300, RHS fender-300, Rear Bumper-..."
35,2022-07-23 12:19:42,MH01DR2403,"Bonnet-300, Front Bumper-300, LHS fender-300, ..."
36,2022-07-23 13:00:51,MH01DR5850,"Front Bumper-300, LHS fender-300"
37,2022-07-23 13:02:51,MH01DR4259,RHS quarter Pannel-100
38,2022-07-23 13:03:25,MH01DR7975,"RHS Front door-300, Wind Shield-100"
39,2022-07-23 13:03:49,MH01CV7014,"Front Bumper-300, Rear Bumper-300, LHS fender-300"


In [33]:
allotment_car_no=responses_df.merge(allotment_s_r, on='Car Number',how='left')

# try:
#     allotment_car_no_df = allotment_car_no[(allotment_car_no['Allotment Date'] <= allotment_car_no['Timestamp']) & (allotment_car_no['Timestamp'] <= allotment_car_no['Return Date'])]
#     # allotment_car_no_df=allotment_car_no_df[['Car Number','ETM']]
#     allotment_car_no_df
# except:
#     allotment_car_no['Timestamp']=pd.to_datetime(allotment_car_no['Timestamp']).dt.strftime('%Y-%m-%d')
#     allotment_car_no_df = allotment_car_no[(allotment_car_no['Allotment Date'] <= allotment_car_no['Timestamp']) & (allotment_car_no['Timestamp'] <= allotment_car_no['Return Date'])]
#     # allotment_car_no_df=allotment_car_no_df[['Car Number','ETM']]
#     allotment_car_no_df
# allotment_car_no_df

,Timestamp,Car Number,Allotment Date,Return Date,ETM,DM Name
5,2022-07-06 18:45:57,MH01DR2761,2022-02-15 19:24:48,2022-07-25 13:45:58,Leasing,Roaring Lions
14,2022-07-07 16:34:59,MH01DR5671,2022-07-05 15:01:57,2022-07-15 18:04:15,ETM14248,Black Panthers
17,2022-07-07 19:21:06,MH01CV7902,2021-08-30 14:56:00,2022-07-25 13:45:58,Leasing,Leasing
27,2022-07-08 12:10:36,MH01DR7075,2022-06-27 17:25:54,2022-07-08 19:02:55,ETM22327,Terrific Tigers
36,2022-07-08 15:05:51,MH01DR3899,2022-06-18 13:20:03,2022-07-08 17:10:52,ETM20251,Terrific Tigers
43,2022-07-09 11:32:01,MH01DR2329,2021-12-20 11:26:20,2022-07-25 13:45:58,Leasing,Black Panthers
49,2022-07-09 12:40:56,MH01DR4159,2022-05-23 15:34:24,2022-07-09 17:35:49,ETM18060,Black Panthers
60,2022-07-09 18:16:44,MH01DR6854,2022-07-09 15:36:42,2022-07-14 17:47:15,Insurance,Deep Hunters
68,2022-07-11 15:13:43,MH01DR2687,2022-05-27 17:20:30,2022-07-25 13:45:58,Leasing,Deadly Shark
80,2022-07-12 18:40:14,MH01DR5689,2022-07-10 18:52:21,2022-07-19 14:53:45,ETM22959,Terrific Tigers


In [35]:
# allotment_car_no_df['Timestamp']=pd.to_datetime(allotment_car_no_df['Timestamp']).dt.strftime('%Y-%m-%d')
# allotment_car_no_df.head(10)
allotment_car_no['Timestamp']=pd.to_datetime(allotment_car_no['Timestamp']).dt.strftime('%Y-%m-%d')
allotment_car_no

,Timestamp,Car Number,Allotment Date,Return Date,ETM,DM Name
0,2022-07-06,MH01DR2761,2021-09-27 16:58:00,2021-12-10 20:47:14,ETM06984,Deep Hunters
1,2022-07-06,MH01DR2761,2021-12-11 11:45:14,2021-12-27 16:57:46,ETM13482,Deep Hunters
2,2022-07-06,MH01DR2761,2021-12-28 10:38:03,2022-01-20 17:35:13,ETM14799,Silent Killers
3,2022-07-06,MH01DR2761,2022-01-22 16:39:00,2022-01-29 17:22:12,ETM15909,Silent Killers
4,2022-07-06,MH01DR2761,2022-01-31 12:36:28,2022-02-15 10:45:18,ETM15339,Deep Hunters
...,...,...,...,...,...,...
75827,NaN,,NaT,2022-07-25 13:45:58,,
75828,NaN,,NaT,2022-07-25 13:45:58,,
75829,NaN,,NaT,2022-07-25 13:45:58,,
75830,NaN,,NaT,2022-07-25 13:45:58,,


In [ ]:
# importing libraries

import pandas as pd
from pandas import Series,DataFrame
import datetime
from datetime import date, timedelta
import pygsheets
import warnings
import numpy as np
warnings.filterwarnings("ignore")

clients = pygsheets.authorize(service_file='repair.json')

allotment_status_report='1cpR6AVVpk9TF4_I38IFYPPOqk-_bSROHgVVYdaXLXOI'
car_no_master_list='11D8_6u4ywy3yNYyrnMonyti6eflqUtpMfsxgLMBXEzk'
penalty_form_repair = '1Txa0MVP7Kxxjhwjd5EwME3JfEPSi5q2klC7GuZw2hRc'

#car no master list

sheet= clients.open_by_key(car_no_master_list)
ws= sheet.worksheet_by_title('History')
data = ws.get_all_values()
headers = data.pop(0)
car_no_master_list = pd.DataFrame(data,columns=headers)
car_no_master_list=car_no_master_list[['Date', 'Car No','DM','End']]
car_no_master_list['Date']=pd.to_datetime(car_no_master_list['Date'], errors='coerce',format='%d-%b-%Y')
car_no_master_list['End']=pd.to_datetime(car_no_master_list['End'], errors='coerce',format='%d/%m/%Y')
car_no_master_list[['Date','End']]=car_no_master_list[['Date','End']].astype(str)
car_no_master_list['Date'].replace('NaT','', inplace=True,regex=True)
car_no_master_list['End'].replace('NaT','', inplace=True,regex=True)
car_no_master_list.rename({'Car No':'Car Number'},axis=1,inplace=True)

In [ ]:
#allotment status report sheet

sheet = clients.open_by_key(allotment_status_report)
ws = sheet.worksheet_by_title('New Allotment History')
data = ws.get_all_values()
headers = data.pop(0)
allotment_s_r = pd.DataFrame(data,columns=headers)
allotment_s_r = allotment_s_r[['Allotment Date','Return Date','Car Number','ETM','DM Name']]
allotment_s_r['Car Number']=allotment_s_r['Car Number'].str.strip()
today = datetime.datetime.now()
today_timestmp_column = today.strftime("%d/%m/%Y %H:%M:%S")
allotment_s_r['Allotment Date']=allotment_s_r['Allotment Date'].replace('',(today_timestmp_column))
allotment_s_r['Return Date']=allotment_s_r['Return Date'].replace('',(today_timestmp_column))
allotment_s_r[['Allotment Date','Return Date']]=allotment_s_r[['Allotment Date','Return Date']].apply(pd.to_datetime, errors='coerce',format='%d/%m/%Y %H:%M:%S')
allotment_s_r[['Allotment Date','Return Date']]=allotment_s_r[['Allotment Date','Return Date']].astype(str)
allotment_s_r['Allotment Date'].replace('NaT','', inplace=True,regex=True)
allotment_s_r['Return Date'].replace('NaT','', inplace=True,regex=True)

In [ ]:
#penalty form sheet

sheet= clients.open_by_key(penalty_form_repair)
ws= sheet.worksheet_by_title('Form responses')
data = ws.get_all_values()
headers = data.pop(0)
responses_df = pd.DataFrame(data,columns=headers)
responses_df=responses_df[['Timestamp', 'Car Number', 'Panel Types']]
responses_df['Car Number']=responses_df['Car Number'].str.strip()
responses_df['Timestamp']=pd.to_datetime(responses_df['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')
responses_df['Timestamp']=responses_df['Timestamp'].astype(str)
responses_df['Timestamp'].replace('NaT','', inplace=True,regex=True)

In [ ]:
allotment_car_no=responses_df.merge(allotment_s_r, on='Car Number',how='left')
allotment_car_no

In [ ]:
allotment_car_no[['Allotment Date','Return Date','Timestamp']]=allotment_car_no[['Allotment Date','Return Date','Timestamp']].apply(pd.to_datetime, errors='coerce',format='%Y-%m-%d %H:%M:%S')
allotment_car_no

In [ ]:
allotment_car_no.dtypes

In [ ]:
allotment_car_no['Timestamp'].nunique()

In [ ]:
allotment_car_no_df = allotment_car_no[(allotment_car_no['Allotment Date'] <= allotment_car_no['Timestamp'])& (allotment_car_no['Timestamp'] <= allotment_car_no['Return Date'])]
allotment_car_no_df

In [ ]:
allotment_car_no_df['Timestamp'].nunique()

In [3]:
pd.set_option('display.max_rows',None)